## Introduction to Chains using the LangChain Package

LangChain is a powerful open-source Python framework that simplifies the development of applications powered by LLMs. It provides a standard interface and a rich set of features to help developers and researchers create, experiment with, and analyze language models and agents.

https://www.langchain.com/

The concept of an "chain" is an important tool for understanding and building generative AI applications. Chains are a key concept in LangChain, as they allow developers to create complex workflows by chaining together various components, such as language models, data sources, and processing steps.

### Table of Contents <a name="top"></a>
1. [Create a LLM inside of LangChain](#llm)
2. [Prompt Templates](#template)
3. [How to chain 2 tasks together](#2chain)
4. [Output Parser](#parser)
5. [How to chain 3 tasks together](#3chain)

This content was adapted from: https://python.langchain.com/docs/get_started/quickstart/


In [1]:
# Remember, in order to run this notebook, you have to run the notebook M2-8a-Config_SM_image to configure the 
# SageMaker Docker image everytime you stop and restart the Jupyterlab Space.
from dotenv import load_dotenv
import os
import langchain

In [2]:
# Configure your API keys
#
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
#
# Not needed for this notebook
# langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
# openai_api_key = os.getenv('OPENAI_API_KEY')
# huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY'
# You can always just assign your variable directly, just not good practice to expose your key in a notebook
# anthropic_api_key='sk-ant-api03....._AAA' 

## Create a LLM model inside LangChain <a name="llm"></a>
Let's start with creating a chat model. We can use one from a wide selection, some of which will be familiar to you.

https://python.langchain.com/docs/integrations/chat/

In [3]:
# Docs: https://api.python.langchain.com/en/latest/chat_models/langchain_anthropic.chat_models.ChatAnthropic.html

from langchain_anthropic import ChatAnthropic

claude_llm = ChatAnthropic(model="claude-3-sonnet-20240229",
                           api_key=anthropic_api_key
                          )
# Show some details about the model
claude_llm.dict()

{'model': 'claude-3-sonnet-20240229',
 'max_tokens': 1024,
 'temperature': None,
 'top_k': None,
 'top_p': None,
 'model_kwargs': {},
 'streaming': False,
 'max_retries': 2,
 'default_request_timeout': None,
 '_type': 'anthropic-chat'}

#### Many other chat models are possible in LangChain
OpenAI Chat Model: https: https://python.langchain.com/docs/integrations/platforms/openai/<BR>
HuggingFace Chat Model: https://python.langchain.com/docs/integrations/chat/huggingface/

In [4]:
# Create a simple prompt using a string
raw_input = "Who is Socrates and why is he relevant to effective teaching?"

In [5]:
# Invoke the model with our prompt (This uses your API KEY to call the model)
response = claude_llm.invoke(raw_input, max_tokens = 1024)
#
# The output from the LLM is an object called AIMessage
print("What is the type of response?:", type(response))

What is the type of response?: <class 'langchain_core.messages.ai.AIMessage'>


In [6]:
# Here is the documentation:
# https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html
#
# Just dump the AIMessage to the screen and notice is is more than just text
response

AIMessage(content="Socrates (470-399 BC) was an ancient Greek philosopher who is considered one of the founders of Western philosophy. He is highly relevant to effective teaching for several reasons:\n\n1. The Socratic Method: Socrates developed a teaching method based on asking thought-provoking questions to stimulate critical thinking and draw out ideas from his students. This method, known as the Socratic method, encourages active learning through dialogue and questioning rather than passive reception of information.\n\n2. Emphasis on Questioning: Socrates believed that questioning was central to learning and understanding. By asking probing questions, he challenged his students to examine their assumptions, justify their beliefs, and arrive at their own conclusions through reason and logic.\n\n3. Focus on Conceptual Understanding: Instead of merely transmitting factual information, Socrates aimed to develop a deeper conceptual understanding in his students. He wanted them to grasp 

In [7]:
# We can also get a Python dictionary
response.dict()

{'content': "Socrates (470-399 BC) was an ancient Greek philosopher who is considered one of the founders of Western philosophy. He is highly relevant to effective teaching for several reasons:\n\n1. The Socratic Method: Socrates developed a teaching method based on asking thought-provoking questions to stimulate critical thinking and draw out ideas from his students. This method, known as the Socratic method, encourages active learning through dialogue and questioning rather than passive reception of information.\n\n2. Emphasis on Questioning: Socrates believed that questioning was central to learning and understanding. By asking probing questions, he challenged his students to examine their assumptions, justify their beliefs, and arrive at their own conclusions through reason and logic.\n\n3. Focus on Conceptual Understanding: Instead of merely transmitting factual information, Socrates aimed to develop a deeper conceptual understanding in his students. He wanted them to grasp the un

In [8]:
# Let's extract just the text using the .content property:
print(response.content)

Socrates (470-399 BC) was an ancient Greek philosopher who is considered one of the founders of Western philosophy. He is highly relevant to effective teaching for several reasons:

1. The Socratic Method: Socrates developed a teaching method based on asking thought-provoking questions to stimulate critical thinking and draw out ideas from his students. This method, known as the Socratic method, encourages active learning through dialogue and questioning rather than passive reception of information.

2. Emphasis on Questioning: Socrates believed that questioning was central to learning and understanding. By asking probing questions, he challenged his students to examine their assumptions, justify their beliefs, and arrive at their own conclusions through reason and logic.

3. Focus on Conceptual Understanding: Instead of merely transmitting factual information, Socrates aimed to develop a deeper conceptual understanding in his students. He wanted them to grasp the underlying principles

[Top of Page](#top)
## Prompt Templates <a name="template"></a>
We have a model ready to use. Next, let's learn about a prompt template.

Prompt templates are predefined recipes for generating prompts for language models. A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task. LangChain provides tooling to create and work with prompt templates. 
LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models. Typically, language models expect the prompt to either be a string or else a list of chat messages.

In [9]:
from langchain.prompts import PromptTemplate

# Create the prompt object
template = PromptTemplate(
    input_variables=["adjective","topic"],
    template="What is a {adjective} joke about {topic}?",
)
# Check out the prompt schema (or configuration)
template.input_schema()

PromptInput(adjective=None, topic=None)

In [10]:
# We can use this template by passing the parameters
template.format(adjective="funny",topic="dogs") # Pass paramaters

'What is a funny joke about dogs?'

In [11]:
# But we can also use the invoke() method and pass a dictionary. 
# This is much more common, use a dictionary to pass the arguments
template.invoke({"adjective":"strange","topic": "bears"}) # Dictionary instead of paramters

StringPromptValue(text='What is a strange joke about bears?')

A prompt template is a way to create consistent prompts, leverage variable passing in Python.

[Top of Page](#top)
## Chains: How to chain 2 tasks together <a name="2chain"></a>
A LangChain **chain** is a fundamental concept in the LangChain framework that allows developers to create  workflows by chaining together various components in to processing steps.

In LangChain, a chain refers to a sequence of steps or components that work together to accomplish a specific task. These components can include:

1. Language Models (LLMs): The main AI models that generate text, such as GPT-3, Cohere, or Hugging Face models.
2. Prompts: The input prompts that are fed into the LLMs to generate the desired output.
3. Parsers: Component that convert the format of an output into another format.
4. Memory: Components that store and retrieve relevant information to maintain context across multiple steps.
5. Agents: Intelligent agents that can dynamically select and use the appropriate tools (e.g., search engines, calculators) to solve a given task.
6. Tools: External components that can be integrated into the chain, such as APIs, databases, or other services.

Chains allow you to go beyond just a single API call to a language model and instead chain together multiple calls in a logical sequence.

In [12]:
# So far, we have two LangChain objects:
print(type(claude_llm))
print(type(template))

<class 'langchain_anthropic.chat_models.ChatAnthropic'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>


In [13]:
# Define a chain by linking the two objects in sequence.
# Discuss the '|' character
chain = template | claude_llm 
type(chain)

langchain_core.runnables.base.RunnableSequence

In [14]:
# To use the chain, use the invoke() method
# Define an adjective and topic
adj = "happy"
top = "pumpkins"
#
# We start the chain by using the invoke() method and passing a dictionary
#
response = chain.invoke({"adjective":adj,"topic": top})
# This output is a combination of both the prompt template and the LLM inferencing the prompt
print(response.content)

Here's a happy, silly joke about pumpkins:

Why did the pumpkin cross the road? To get to the other side for the harvest festival!


So far, we have used a 2-step chain.  Let's work towards a 3-step chain.

[Top of Page](#top)
## Third LangChain Object: Parser <a name="parser"></a>
Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format. This is very useful when you are using LLMs to generate any form of structured data.

In [15]:
from langchain_core.output_parsers import StrOutputParser

# This parser takes the AIMessage reutrned form the LLM and converts it to a string
output_parser = StrOutputParser()

In [16]:
# Mannually Invoke the LLM to get a response.
response = claude_llm.invoke("Tell me a funny joke.", max_tokens=100)
#
# Check the type of the response: AIMessage
print('Check out the type of the return from the LLM:', type(response), '\n')
#
# Let's use the output_parser to convert the AIMessage to a string
parsed_response = output_parser.invoke(response)
#
# Check the type, it should be a string
print('After the parser, the output is of type:', type(parsed_response), '\n')
#
# Print the string
print(parsed_response)

Check out the type of the return from the LLM: <class 'langchain_core.messages.ai.AIMessage'> 

After the parser, the output is of type: <class 'str'> 

Here's a silly joke for you:

Why can't a bicycle stand up by itself?
Because it's two-tired!


[Top of Page](#top)
## Chains: How to chain 3 tasks together  <a name="3chain"></a>
Now we have 3 objects: A prompt template, a LLM and an output parser

In [17]:
print(type(template))
print(type(claude_llm))
print(type(output_parser))

<class 'langchain_core.prompts.prompt.PromptTemplate'>
<class 'langchain_anthropic.chat_models.ChatAnthropic'>
<class 'langchain_core.output_parsers.string.StrOutputParser'>


In [18]:
# Create a chain with 3 steps
chain = template | claude_llm | output_parser
type(chain)

langchain_core.runnables.base.RunnableSequence

In [19]:
# Again, use the chain by sending the dictionary to the first prompt template
# Define an adjective and topic
adj = "witty"
top = "carrots"
#
# We start the chain by using the invoke() method.
response = chain.invoke({"adjective":adj,"topic":top})
# Check out the response now, already converted to a string.
print("Type of the resopnse:", type(response))
print(response)

Type of the resopnse: <class 'str'>
Here's a witty carrot joke:

Why didn't the carrot go pro? Because he wasn't a-maize-ing enough.


## What we did:
1. Created a LLM object inside of LangChain
2. Created a prompt template object
3. Created a 2-step chain and used it (prompt + llm)
4. Created an ouput parser to convert AIMessage to a string
5. Created a 3-step chain and used it (prompt + llm + output parser)

[Top of Page](#top)